In [ ]:
from pathlib import Path
import os
import re
import datetime
import warnings
from shutil import copyfile
from dataclasses import dataclass
import json
import pandas as pd
import numpy as np

BACKUP_DIR = Path("./backups")
UPOWER_BACKUP_DIR = BACKUP_DIR / "upower"



In [ ]:
json_file_names = os.listdir(UPOWER_BACKUP_DIR)
data = []
for filename in json_file_names:
    with open(UPOWER_BACKUP_DIR / filename, "r", encoding="utf8") as f:
        data.append(json.load(f))
len(json_file_names)
df = pd.DataFrame(data).sort_values("created_utc")
df['date'] = df['created_utc'].transform(datetime.datetime.fromtimestamp)
df.sort_values("created_utc").iloc[-1]['date']

In [ ]:
df.set_index("created_utc")['energy'].plot()

In [ ]:
df.set_index("created_utc")['energy_rate'].plot()

In [ ]:
df['created_utc'].sort_values()

In [ ]:
import datetime

In [ ]:
data = [
    {
        "description": "Discharge at constant power consumption. (No work done at pc)",
        "min_ts": 1706085300,
        "max_ts": 1706094734,
    },
    {
        "description": "Zugfahrt Basel - Zuerich, kein Docker Container und kein Notebook am laufen",
        "min_ts": 1707023500,
        "max_ts": 1707026694,
    },
    {
        "description": "Discharge Run 1",
        "min_ts": 1708816000,
        "max_ts": 1708819050,
    },
    {
        "description": "Discharge Run 2",
        "min_ts": 1708820000,
        "max_ts": 1708820000 + 3200,
    }
]

idx = 3

min_ts = data[idx]['min_ts']
max_ts = data[idx]['max_ts']
check_df = df.loc[(df['created_utc'] >= min_ts) & (df['created_utc'] <= max_ts)]

check_df.set_index("created_utc")['energy'].plot()
check_df.set_index("created_utc")['energy_rate'].plot()


print(f"Period from '{datetime.datetime.fromtimestamp(check_df['created_utc'].min())}' ({check_df['created_utc'].min()}) to"
      f" '{datetime.datetime.fromtimestamp(check_df['created_utc'].max())}'")

In [ ]:
from matplotlib import pyplot as plt

fig, axs = plt.subplots(len(data))

for idx in range(len(data)):
    min_ts = data[idx]['min_ts']
    max_ts = data[idx]['max_ts']
    check_df = df.loc[(df['created_utc'] >= min_ts) & (df['created_utc'] <= max_ts)]
    check_df.set_index("created_utc")['energy'].plot(ax=axs[idx])
    check_df.set_index("created_utc")['energy_rate'].plot(ax=axs[idx])

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
model = LinearRegression()

In [ ]:
y = check_df.set_index("created_utc")['energy']
x = np.array(check_df['created_utc']).reshape((-1, 1))
model = model.fit(x, y)
model.coef_